In [ ]:
# from mimic_cxr_jpg_loader import MIMIC_CXR_JPG_Loader
import os
import gzip
import pandas as pd

data_folder = '../data/physionet.org/files/mimic-cxr-jpg/2.0.0/files'
csv_folder = '../data/physionet.org/files/mimic-cxr-jpg/2.0.0'
metadata_csv_file = 'mimic-cxr-2.0.0-metadata.csv.gz'
split_csv_file = 'mimic-cxr-2.0.0-split.csv.gz'
chexpert_csv_file = 'mimic-cxr-2.0.0-chexpert.csv.gz'

In [ ]:
# Read .csv info files
with gzip.open(os.path.join(csv_folder, chexpert_csv_file), 'rt') as file:
    label_csv = pd.read_csv(file)
with gzip.open(os.path.join(csv_folder, metadata_csv_file), 'rt') as file:
    metadata_csv = pd.read_csv(file)
with gzip.open(os.path.join(csv_folder, split_csv_file), 'rt') as file:
    split_csv = pd.read_csv(file)
    
# Merge the data
merged_data = pd.merge(metadata_csv, split_csv, on=['dicom_id', 'study_id', 'subject_id'])
grouped_data = merged_data.groupby('split', group_keys=False)
grouped_data.get_group('validate')

In [1]:
#################### Test dataset loader ####################
from mimic_cxr_jpg_loader import MIMIC_CXR_JPG_Loader

myCustomDataLoader = MIMIC_CXR_JPG_Loader({'train': 10, 'validate': 10, 'test': 10})
train_dataset, val_dataset, test_dataset = myCustomDataLoader.load()

# Accessing information
num_train_images = train_dataset.cardinality().numpy()
# num_classes = myCustomDataLoader.label_csv.shape[1] - 2  # excluding subject_id and study_id

# Accessing images and labels
for image, label, info in train_dataset.take(1):
    print("Image Shape:", image.shape)
    print("Label:", label)
    print("Info:", info)

2024-01-12 09:57:52.608353: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
import tensorflow as tf

# Example dataset creation
dataset = tf.data.Dataset.range(10)  # Creating a simple dataset of integers from 0 to 9

# Specify the index of the sample you want to retrieve
sample_index = 3

# Skip the samples before the desired index
dataset = dataset.skip(sample_index)

# Take only the next sample after skipping
desired_sample = dataset.take(1)

# Iterate over the dataset to retrieve the desired sample
for sample in desired_sample:
    print(sample.numpy())